In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from skimage.measure import label,regionprops
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import json
import nibabel as nib
import tensorflow as tf
import cv2
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import pandas as pd

In [3]:
labels = ["0-2", "3-9", "10-20", "21-25", "26-27", "28-31", "32-36", "37-45", "46-54", "55-65", "66-116"]
train_data = []
train_labels = []
test_data = []
test_labels = []
EPOCHS = 50
BS = 512

In [4]:
train_df = pd.read_csv("/content/drive/My Drive/csvs/images_filenames_labels_train_augmented.csv")
test_df = pd.read_csv("/content/drive/My Drive/csvs/images_filenames_labels_test.csv")

In [5]:
train_df["target"] = train_df["target"].astype("string")
test_df["target"] = test_df["target"].astype("string")

In [6]:
aug = ImageDataGenerator()

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(200,200,1)),
    tf.keras.layers.Conv2D(32, 3, activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, 3, activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, 3, activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, 2, activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, 2, activation = 'relu', kernel_initializer = 'he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),                          
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(11, activation='softmax') 
])


In [ ]:
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 198, 198, 32)      320       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 99, 99, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 99, 99, 32)        128       
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 48, 48, 64)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 46, 46, 128)      

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-4), loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.save_weights("/content/drive/My Drive/ages.h5")

In [9]:
model.load_weights("/content/drive/My Drive/age_checkpoint")

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/My Drive/age_checkpoint",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.fit(aug.flow_from_dataframe(train_df, x_col='filename', y_col='target', target_size=(200,200), color_mode='grayscale', class_mode="sparse", batch_size=BS),
     validation_data=aug.flow_from_dataframe(test_df, x_col='filename', y_col='target', target_size=(200,200), color_mode='grayscale', class_mode="sparse", batch_size=BS),  epochs=EPOCHS,  callbacks=[model_checkpoint_callback])

Found 234400 validated image filenames belonging to 11 classes.
Found 10046 validated image filenames belonging to 11 classes.
Epoch 1/50
458/458 [==============================] - 484s 1s/step - loss: 1.8908 - accuracy: 0.2976 - val_loss: 1.5772 - val_accuracy: 0.4030
Epoch 2/50
458/458 [==============================] - 475s 1s/step - loss: 1.5527 - accuracy: 0.3960 - val_loss: 1.4021 - val_accuracy: 0.4634
Epoch 3/50
458/458 [==============================] - 477s 1s/step - loss: 1.4115 - accuracy: 0.4448 - val_loss: 1.2401 - val_accuracy: 0.5219
Epoch 4/50
458/458 [==============================] - 477s 1s/step - loss: 1.3043 - accuracy: 0.4844 - val_loss: 1.1378 - val_accuracy: 0.5655
Epoch 5/50
458/458 [==============================] - 482s 1s/step - loss: 1.2104 - accuracy: 0.5198 - val_loss: 1.0627 - val_accuracy: 0.5984
Epoch 6/50
458/458 [==============================] - 487s 1s/step - loss: 1.1184 - accuracy: 0.5570 - val_loss: 0.9704 - val_accuracy: 0.6464
Epoch 7/50
458/

In [ ]:
model.evaluate(aug.flow_from_dataframe(test_df, x_col='filename', y_col='target', target_size=(200,200), color_mode='grayscale', class_mode="sparse", batch_size=BS))

Found 10046 validated image filenames belonging to 11 classes.
20/20 [==============================] - 65s 2s/step - loss: 0.7248 - accuracy: 0.8913


[0.7248237133026123, 0.8913000226020813]